# Intro

The OECD database is composed of hundreds (if not thousands) of tables that focus on specific economic and social variables. These variables are attirbutable to individual countries, and are generally recorded over time, making them a time series. 

For the purposes of this analysis, we are only interested in the latest year's data point, which would reflect the most recent characterization of each country's social and economic situation. 

However, given that we have years of data for each country, there is also the potential to use the change in the variable as a feature of our modeling. For example, rapid improvement in immunization coverage for a country would imply effective health education and could represent a more accurate picture of how the country is better educating the public about a new pandemic versus a country with a higher, but stagnant, immunization record. 

For now, we are combining the various OECD tables on the 'country' key so that we can merge it with COVID data.

Data transformation to follow.

# Loading and Cleaning Data

Here we load and clean csv data taken from the OECD database. This includes datasets on the following topics:
- Education
- Government Debt
- Immunization
- Demographics / Population
- Tourism
- Wealth Distribution

Here we load and clean json data taken from the COVID-19 database. This includes datasets on the following topics:
- Education
- Government Debt
- Immunization
- Demographics / Population
- Tourism
- Wealth Distribution

For each dataset, there are multiple columns we are not interested in. Furthermore, for the columns we do want, there are various version of that variable, so we need to filter on what we are looking for. The tables are not standardized, so each csv file will have to be cleaned and prepared individually. 

### Load Packages

In [1]:
import pandas as pd
from urllib.request import urlopen
import ssl
import json
import requests

ssl._create_default_https_context = ssl._create_unverified_context

## OECD
### Education

We hypothesize that a country's education should play a key role in the success rate of the country's approach to COVID. Specifically, a more highly educated country should, in theory, have a more effective approach to dealing with the virus, and should have a populace that better understands public health terminiology, what viruses are, and would be more willing to take the pandemic seriously. 

So we will focus on the share of population of countries that have a tertiary education level. This is defined on wikipedia as:

*Tertiary education, also referred to as third-level, third-stage or post-secondary education, is the educational level following the completion of secondary education. The World Bank, for example, defines tertiary education as including universities as well as trade schools and colleges.*

In [15]:
url_edu = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Education_Statistics.csv'
dfraw_edu = pd.read_csv(url_edu)


In [16]:
columns_to_use = ['COUNTRY', 'Country', 'Gender', 'ISCED 2011 A education level', 'Reference Period', 'Measure', 'Value']

# Only select the columns we want to use
df_edu = dfraw_edu[columns_to_use]

# filter some of the columns to include the variables we want to see
df_edu = df_edu[df_edu['ISCED 2011 A education level'] == "Tertiary education"]
df_edu = df_edu[df_edu['Gender'] == "Total"]
df_edu = df_edu[df_edu['Measure'] == "Value"]

# Choose the latest datapoint for each country
df_edu = df_edu.sort_values('Reference Period').groupby('Country').tail(1)

# This final dataframe contains the share of each country's population that has a tertiary education. 
df_edu.head()


,COUNTRY,Country,Gender,ISCED 2011 A education level,Reference Period,Measure,Value
3244,CHN,China (People's Republic of),Total,Tertiary education,2010.0,Value,9.681174
3302,IND,India,Total,Tertiary education,2011.0,Value,10.595189
2846,SAU,Saudi Arabia,Total,Tertiary education,2016.0,Value,23.623922
2772,CHL,Chile,Total,Tertiary education,2017.0,Value,25.168179
2757,RUS,Russia,Total,Tertiary education,2017.0,Value,56.700150


In [4]:
# But lets reduce this table down to what we'll be combining together later.
# A simple country-variable table.

df_edu = df_edu[['COUNTRY', 'Country', 'Value']]
df_edu.columns = ['ISO', 'Country', 'Tertiary_Education_Pct']
df_edu.head()

,ISO,Country,Tertiary_Education_Pct
447,KOR,Korea,49.008511
701,CAN,Canada,57.888363
709,JPN,Japan,51.928062
774,CZE,Czech Republic,24.262077
909,FRA,France,36.897491


### Debt

In [17]:
# Load the Debt Data
url_debt = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Government_Debt.csv'
dfraw_debt = pd.read_csv(url_debt)

In [30]:
# Choose the latest data point for each country
df_debt = dfraw_debt.sort_values('Time period').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_debt = df_debt[['COU', 'Country', 'Value']]
df_debt.columns = ['ISO', 'Country', 'Debt_to_GDP']

# Check to make sure it looks ok
df_debt.head()

,ISO,Country,Gov_Debt_Pct_GDP
208,JPN,Japan,183.530
278,NZL,New Zealand,30.450
264,NLD,Netherlands,51.845
320,PRT,Portugal,87.962
27,AUT,Austria,65.754


### Immunization

In [21]:
# Load Immunization Data
url_imm = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Immunization_Statistics.csv'
dfraw_imm = pd.read_csv(url_imm)


In [31]:
# Choose the latest data point for each country
df_imm = dfraw_imm.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_imm = df_imm[['COU', 'Country', 'Value']]
df_imm.columns = ['ISO', 'Country', 'Pct_65_Immunized_Flu']

# Check to make sure it looks ok
df_imm.head()

,ISO,Country,Pct_Immunized_Flu
6,AUS,Australia,74.6
351,CHE,Switzerland,46.0
8,AUT,Austria,20.3
271,POL,Poland,9.7
224,MEX,Mexico,82.3


### Population / Demographics

In [33]:
# Load the Demographics Data
url_pop = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Population_Statistics.csv'
dfraw_pop = pd.read_csv(url_pop)

In [39]:
# Create an old age ratio dataframe
df_oldage = dfraw_pop[dfraw_pop['AGE'] == "65_OVER_SHARE"]
df_oldage = df_oldage[df_oldage['Sex'] == "Total"]

# Choose the latest data point for each country
df_oldage = df_oldage.sort_values('Time').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_oldage = df_oldage[['LOCATION', 'Country', 'Value']]
df_oldage.columns = ['ISO', 'Country', 'Old_Age_Ratio']

# Check to make sure it looks ok
df_oldage.head()

,ISO,Country,Old_Age_Ratio
58757,EU28,European Union (28 countries),19.849
58715,G20,G20,11.037
58673,WLD,World,8.921
57623,TUR,Turkey,8.649
58589,SVN,Slovenia,19.670


In [ ]:
# Create a sex ratio dataframe
df_sexratio = dfraw_pop[dfraw_pop['AGE'] == "TOTAL"] #i.e. total population of the country

# STILL NEED TO DO

### Tourism

In [40]:
url_tour = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Tourism_Statistics.csv'
dfraw_tour = pd.read_csv(url_tour)


In [50]:
dfraw_tour.head()
# Create Toursim as % of GDP dataframe
df_tour_gdp = dfraw_tour[dfraw_tour['KEY_IND_PC'] == "KEYIND_TOURGDPPC"]

# Choose the latest data point for each country
df_tour_gdp = df_tour_gdp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_tour_gdp = df_tour_gdp[['COUNTRY', 'Country', 'Value']]
df_tour_gdp.columns = ['ISO', 'Country', 'Tourism_Pct_GDP']

# Check to make sure it looks ok
df_tour_gdp.head()



,ISO,Country,Tourism_Pct_GDP
564,MLT,Malta,6.129
119,IND,India,3.759
108,EGY,Egypt,4.258
115,EST,Estonia,5.428
66,POL,Poland,1.175


In [55]:
# Create Toursim employment as % of Total
df_tour_emp = dfraw_tour[dfraw_tour['KEY_IND_PC'] == "KEYIND_TOUREMPDIRECTPC"]

# Choose the latest data point for each country
df_tour_emp = df_tour_emp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_tour_emp = df_tour_emp[['COUNTRY', 'Country', 'Value']]
df_tour_emp.columns = ['ISO', 'Country', 'Tourism_Pct_Employment']

# Check to make sure it looks ok
df_tour_emp.head()

,ISO,Country,Tourism_Pct_Employment
387,ARG,Argentina,10.17
460,IND,India,5.31
465,IDN,Indonesia,8.37
577,PER,Peru,3.82
282,ITA,Italy,8.30


### Wealth Distribution

In [56]:
url_wealth = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Wealth_Distribution_Statistics.csv'
dfraw_wealth = pd.read_csv(url_wealth)

In [58]:
# Create two dataframes -  share of top 1% wealth, and share of bottom 60% of wealth
dfraw_wealth.Variable.unique()

array(['Mean financial assets per household (current prices)',
       'Mean liabilities per household (current prices)',
       'Mean net wealth per household (current prices)',
       'Mean net wealth per person (current prices)',
       'Mean to median net wealth ratio',
       'Median debt-to-assets ratio of indebted households',
       'Median debt-to-income ratio of indebted households',
       'Median net wealth per household (current prices)',
       'Share of indebted households with debt-to-assets ratio above 75%',
       'Share of bottom 60% of wealth', 'Share of indebted households',
       'Share of indebted households with debt-to-income ratio above 3',
       'Share of individuals with eq. liquid financial wealth <25% of income poverty line',
       'Share of individuals with eq. liquid financial wealth <50 % of  income poverty line',
       'Share of individuals with eq. net wealth < 25% of income poverty line',
       'Share of individuals with eq. net wealth <50 % of  

### Life Expectancy


In [ ]:
url_life_exp = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Life_Expectancy.csv'
dfraw_life_exp = pd.read_csv(url_life_exp)

In [ ]:
# TODO

## COVID-19

All attempts to load json via web is throwing an SSL error. We will load locally.

-Method 1
-Request fails unless we provide a user-agent
api_response = requests.get('https://thevirustracker.com/timeline/map-data.json', headers={"User-Agent": "Chrome"})
covid_stats = api_response.json()

-Method 2
response = urlopen('https://thevirustracker.com/timeline/map-data.json')
json_data = response.read().decode('utf-8', 'replace')
df = json.loads(json_data)

### Country Covid-19 Stats

In [ ]:
#Importing COVID19 Global Data
country_stats = pd.read_json(r'covid_full.json')

In [ ]:
lst_1=[]
for i in country_stats['data']:
    for key,value in i.items():
        pair=[key,value]
        lst_1.append(pair)

In [ ]:
pd.DataFrame((lst_1), columns =['Stat', 'Value']) 

,Stat,Value
0,countrycode,AD
1,date,6/06/20
2,cases,852
3,deaths,51
4,recovered,1
5,countrycode,AD
6,date,6/05/20
7,cases,852
8,deaths,51
9,recovered,1


### Global Covid-19 Stats

In [ ]:
#Importing COVID19 Global Data
global_stats = pd.read_json(r'global_stats.json')

In [ ]:
lst=[]
for i in global_stats['results']:
    for key,value in i.items():
        pair=[key,value]
        lst.append(pair)

In [ ]:
pd.DataFrame((lst), columns =['Stat', 'Value']) 

## Combining the OECD Data

For now, we are combining the various OECD tables on the 'country' key so that we can merge it with COVID data.

Data transformation to follow.